<a href="https://colab.research.google.com/github/vishalgimhan/NLP-Projects/blob/main/Hotel_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
#import libraries
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')

import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from ast import literal_eval

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [85]:
#Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Datasets/Hotel_Reviews.csv')
data.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [86]:
data.Hotel_Address.unique()

array([' s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands',
       '1 15 Templeton Place Earl s Court Kensington and Chelsea London SW5 9NB United Kingdom',
       '1 2 Serjeant s Inn Fleet Street City of London London EC4Y 1LL United Kingdom',
       ...,
       'Wilton Place Knightsbridge Westminster Borough London SW1X 7RL United Kingdom',
       'Wrights Lane Kensington and Chelsea London W8 5SP United Kingdom',
       'Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 Vienna Austria'],
      dtype=object)

In [87]:
# Replacing "United Kingdom" with "UK"
data.Hotel_Address = data.Hotel_Address.str.replace("United Kingdom", "UK")

#Split the address and pick the last word to identify the country
data['countries'] = data.Hotel_Address.apply(lambda x: x.split(' ')[-1])

In [88]:
data['countries']

0         Netherlands
1         Netherlands
2         Netherlands
3         Netherlands
4         Netherlands
             ...     
515733        Austria
515734        Austria
515735        Austria
515736        Austria
515737        Austria
Name: countries, Length: 515738, dtype: object

In [89]:
data.columns

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'lat', 'lng', 'countries'],
      dtype='object')

In [90]:
#Drop unecessary columns
data.drop(['Additional_Number_of_Scoring','Review_Date', 'Reviewer_Nationality', 'Negative_Review', 'Review_Total_Negative_Word_Counts',
           'Total_Number_of_Reviews','Positive_Review', 'Review_Total_Positive_Word_Counts', 'Total_Number_of_Reviews_Reviewer_Has_Given',
           'Reviewer_Score','days_since_review', 'lat', 'lng'], axis=1, inplace=True)

In [91]:
data['Tags'][0]

"[' Leisure trip ', ' Couple ', ' Duplex Double Room ', ' Stayed 6 nights ']"

In [92]:
#Convert string representation of the list to a normal list
def impute(column):
  column = column[0]
  if (type(column) != list):
    return "".join(literal_eval(column)) #literal_eval from the ast module to evaluate the string representation of the data
  else:
    return column

data["Tags"] = data[["Tags"]].apply(impute, axis=1)
data.head()

,Hotel_Address,Average_Score,Hotel_Name,Tags,countries
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Couple Duplex Double Room Sta...,Netherlands
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Couple Duplex Double Room Sta...,Netherlands
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Family with young children Dup...,Netherlands
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Solo traveler Duplex Double Ro...,Netherlands
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Couple Suite Stayed 2 nights ...,Netherlands


In [93]:
#Lowercase columns
data['countries'] = data['countries'].str.lower()
data['Tags'] = data['Tags'].str.lower()

In [113]:
#Recommend Hotel and Rank
def recommend_hotel(location, description):
  description = description.lower()

  stop_words = stopwords.words('english') #Retrieves a list of stop words from NLTK
  lemm = WordNetLemmatizer()

  #Creates a set of words from the description excluding stop words
  filtered= {word for word in description if not word in stop_words}
  filtered_set = set()

  for fs in filtered:
    filtered_set.add(lemm.lemmatize(fs)) #reducing each word to their base

  country = data[data['countries'] == location.lower()]

  #Sets the index of the DataFrame country to a range of integers from 0 to the number of rows in the DataFrame.
  country = country.set_index(np.arange(country.shape[0]))

  list1 = []; list2 = []; cos = [];
  for i in range(country.shape[0]):
    temp_token = word_tokenize(country['Tags'][i]) #words extracted from the 'Tags' column
    temp_set = [word for word in temp_token if not word in stop_words]
    temp2_set = set()

    #Lemmatize each word in temp_set
    for s in temp_set:
      temp2_set.add(lemm.lemmatize(s))

    #common words between the hotel's tags and the filtered set of words from the description
    vector = temp2_set.intersection(filtered_set)
    cos.append(len(vector)) #represents the similarity between the hotel's tags and the description

  country['similarity']=cos
  country = country.sort_values(by='similarity', ascending=False)
  country.drop_duplicates(subset='Hotel_Name', keep='first', inplace=True)
  country.sort_values('Average_Score', ascending=False, inplace=True)
  country.reset_index(inplace=True)

  return country[["Hotel_Name", "Average_Score", "Hotel_Address"]].head()

In [102]:
data.head()

,Hotel_Address,Average_Score,Hotel_Name,Tags,countries
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,leisure trip couple duplex double room sta...,netherlands
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,leisure trip couple duplex double room sta...,netherlands
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,leisure trip family with young children dup...,netherlands
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,leisure trip solo traveler duplex double ro...,netherlands
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,leisure trip couple suite stayed 2 nights ...,netherlands


In [114]:
recommend_hotel('Italy', "I am going on a Business Trip")

,Hotel_Name,Average_Score,Hotel_Address
0,Excelsior Hotel Gallia Luxury Collection Hotel,9.4,Piazza Duca D Aosta 9 Central Station 20124 Mi...
1,Palazzo Parigi Hotel Grand Spa Milano,9.3,Corso Di Porta Nuova 1 Milan City Center 20121...
2,Hotel Spadari Al Duomo,9.3,Via Spadari 11 Milan City Center 20123 Milan I...
3,Room Mate Giulia,9.3,Silvio Pellico 4 Milan City Center 20121 Milan...
4,UNA Maison Milano,9.3,Via Mazzini 4 Milan City Center 20123 Milan Italy


In [115]:
recommend_hotel('UK', "I am going on a honeymoon, I need a honeymoon suite room for 3 nights")

,Hotel_Name,Average_Score,Hotel_Address
0,Haymarket Hotel,9.6,1 Suffolk Place Westminster Borough London SW1...
1,41,9.6,41 Buckingham Palace Road Westminster Borough ...
2,Taj 51 Buckingham Gate Suites and Residences,9.5,Buckingham Gate Westminster Borough London SW1...
3,Charlotte Street Hotel,9.5,15 17 Charlotte Street Hotel Westminster Borou...
4,Ham Yard Hotel,9.5,One Ham Yard Westminster Borough London W1D 7D...
